# Estimate Binomial draw probabilility

In [1]:
using DynamicHMCModels

Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-2.6548056865833978

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::BernoulliProblem) =
    as((α = as𝕀, ),  )
# Use a flat priors (the default, omitted) for α
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Sample

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
0.0014 s/step ...done
MCMC, adapting ϵ (25 steps)
9.7e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0025 s/step ...done
MCMC, adapting ϵ (100 steps)
1.0e-5 s/step ...done
MCMC, adapting ϵ (200 steps)
9.1e-6 s/step ...done
MCMC, adapting ϵ (400 steps)
7.1e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
1.0e-5 s/step ...done
MCMC (1000 steps)
1.7e-5 s/step ...done


(NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([1.5398], -3.17689, 1, DoubledTurn, 0.961654, 1), NUTS_Transition{Array{Float64,1},Float64}([1.60264], -3.29407, 1, DoubledTurn, 0.98637, 1), NUTS_Transition{Array{Float64,1},Float64}([2.32971], -4.47452, 1, AdjacentTurn, 0.855739, 3), NUTS_Transition{Array{Float64,1},Float64}([2.09169], -4.44339, 2, DoubledTurn, 1.0, 3), NUTS_Transition{Array{Float64,1},Float64}([1.5596], -4.29099, 2, DoubledTurn, 0.981106, 3), NUTS_Transition{Array{Float64,1},Float64}([0.32772], -4.05034, 1, AdjacentTurn, 0.906731, 3), NUTS_Transition{Array{Float64,1},Float64}([0.706856], -3.2353, 1, DoubledTurn, 1.0, 1), NUTS_Transition{Array{Float64,1},Float64}([0.821846], -3.1253, 2, DoubledTurn, 0.965629, 3), NUTS_Transition{Array{Float64,1},Float64}([0.366699], -3.31416, 1, AdjacentTurn, 0.905129, 3), NUTS_Transition{Array{Float64,1},Float64}([0.818333], -3.1857, 1, AdjacentTurn, 1.0, 3)  …  NUTS_Transition{Array{Float64,1},Floa

To get the posterior for ``α`` use `get_position` and then transform back.

In [7]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));

Extract the parameter.

In [8]:
posterior_α = first.(posterior);

check the effective sample size

In [9]:
ess_α = effective_sample_size(posterior_α)

280.9843009131624

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.92, min/25%/median/75%/max: 0.15 0.89 0.97 1.0 1.0
  termination: AdjacentTurn => 34% DoubledTurn => 66%
  depth: 1 => 65% 2 => 35%


check the mean

In [11]:
mean(posterior_α)

0.7214116805684743

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*